In [1]:
import esm_parser

import logging
logging.basicConfig(level=logging.DEBUG, format=esm_parser.FORMAT)

In [2]:
config = esm_parser.ConfigTest("test_echam")

config

[2019-07-22 14:13:32,352,352:esm_parser.py:74 -    yaml_file_to_dict() ] IOError (2) File not found with /Users/pgierz/Documents/Code/esm_yaml/functions/general_py/..//test_echam/test_echam, trying another extension pattern.
[2019-07-22 14:13:32,354,354:esm_parser.py:74 -    yaml_file_to_dict() ] IOError (2) File not found with /Users/pgierz/Documents/Code/esm_yaml/functions/general_py/..//test_echam/test_echam.yml, trying another extension pattern.
[2019-07-22 14:13:32,370,370:esm_parser.py:151 - attach_to_config_and_remove() ] Attaching: test_echam to datasets.yaml


{}

In [3]:
esm_parser.list_all_keys_starting_with_choose(config)

{}

In [4]:
c = esm_parser.ConfigSetup("test_echam")

[2019-07-22 14:13:32,400,400:esm_parser.py:74 -    yaml_file_to_dict() ] IOError (2) File not found with /Users/pgierz/Documents/Code/esm_yaml/functions/general_py/..//test_echam/test_echam, trying another extension pattern.
[2019-07-22 14:13:32,402,402:esm_parser.py:74 -    yaml_file_to_dict() ] IOError (2) File not found with /Users/pgierz/Documents/Code/esm_yaml/functions/general_py/..//test_echam/test_echam.yml, trying another extension pattern.
[2019-07-22 14:13:32,417,417:esm_parser.py:151 - attach_to_config_and_remove() ] Attaching: test_echam to datasets.yaml
[2019-07-22 14:13:32,434,434:esm_parser.py:861 -         _config_init() ] setup config is being updated with setup_relevant_configs
[2019-07-22 14:13:32,434,434:esm_parser.py:111 - attach_to_config_and_reduce_keyword() ] Attaching:  for test_jsbach
[2019-07-22 14:13:32,435,435:esm_parser.py:74 -    yaml_file_to_dict() ] IOError (2) File not found with /Users/pgierz/Documents/Code/esm_yaml/functions/general_py/..//test_jsba

In [5]:
c.setup_config

{'computer': {'hostnames': {'login': 'ollie[01]',
   'compute': 'prod-[0-9]{3}',
   'mini': 'mini'},
  'nodetypes': ['login', 'compute', 'fat', 'mini'],
  'batch_system': 'slurm',
  'operating_system': {'linux': 'centos'},
  'jobtype': 'compute',
  'choose_jobtype': {'post': {'partition': 'smp'},
   'compute': {'partition': 'mpp'}},
  'choose_partition': {'mpp': {'cores_per_node': 36},
   'smp': {'cores_per_node': 1}},
  'logical_cpus_per_core': 2,
  'threads_per_core': 1,
  'pool_directories': {'pool': '/work/ollie/pool',
   'projects': '/work/ollie/projects'},
  'cores_per_node': 888,
  'choose_submitted': {True: {'modules_needed': ['centoslibs']}},
  'submitted': True,
  'accounting': False,
  'hyper_flag': ''},
 'standalone': True}

In [37]:
c.all_set_variables['computer']['choose_jobtype']

{('computer', 'partition')}

In [35]:
def find_one_independent_choose(all_set_variables):
    task_list = []
    # task_list=['choose_partition', 'choose_jobtype']
    for key in all_set_variables:
        value = all_set_variables[key]
        for choose_keyword, set_vars in value.items():
        #choose_keyword="choose_partition"
        #set_vars=value[choose_keyword]
            task_list.append((key, choose_keyword))
            task_list = add_more_important_tasks(choose_keyword, all_set_variables, task_list)
            logging.debug(task_list)
            return task_list


        
def add_more_important_tasks(choose_keyword, all_set_variables, task_list):
    logging.debug("Incoming task list %s", task_list)
    keyword = choose_keyword.replace("choose_", "")
    logging.debug("Keyword = %s", keyword)
    for model in all_set_variables:
        for choose_thing in all_set_variables[model]:
            logging.debug("Choose_thing = %s", choose_thing)
            for (host, keyword_that_is_set) in all_set_variables[model][choose_thing]:
                logging.debug("Host = %s, keyword_that_is_set=%s", host, keyword_that_is_set)
                if keyword_that_is_set == keyword:
                    if (model, choose_thing) not in task_list:
                        task_list.insert(0, (model, choose_thing))
                        add_more_important_tasks(choose_thing, all_set_variables, task_list)
                        return task_list
                    else:
                        raise KeyError("Opps cyclic dependency: %s" % task_list)
    return task_list
        

In [36]:
find_one_independent_choose(c.all_set_variables)

[2019-07-22 15:07:18,599,599:<ipython-input-35-c492ca057679>:17 - add_more_important_tasks() ] Incoming task list [('computer', 'choose_jobtype')]
[2019-07-22 15:07:18,600,600:<ipython-input-35-c492ca057679>:19 - add_more_important_tasks() ] Keyword = jobtype
[2019-07-22 15:07:18,600,600:<ipython-input-35-c492ca057679>:22 - add_more_important_tasks() ] Choose_thing = choose_jobtype
[2019-07-22 15:07:18,601,601:<ipython-input-35-c492ca057679>:24 - add_more_important_tasks() ] Host = computer, keyword_that_is_set=partition
[2019-07-22 15:07:18,602,602:<ipython-input-35-c492ca057679>:22 - add_more_important_tasks() ] Choose_thing = choose_partition
[2019-07-22 15:07:18,603,603:<ipython-input-35-c492ca057679>:24 - add_more_important_tasks() ] Host = computer, keyword_that_is_set=cores_per_node
[2019-07-22 15:07:18,603,603:<ipython-input-35-c492ca057679>:22 - add_more_important_tasks() ] Choose_thing = choose_submitted
[2019-07-22 15:07:18,604,604:<ipython-input-35-c492ca057679>:24 - add_mo

[('computer', 'choose_jobtype')]

In [24]:
def insert_to_list(l):
    l.insert(0, "a")
    return
    
l = ["a"]
insert_to_list(l)
l

['a', 'a']